# Text Processing (Butter and Cheese Reviews)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kftsu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('butter_cheese_review_ie.csv')

In [3]:
train.head(5)

,Review,Brand,Category
0,"I didn’t get the flavor I was expecting, espec...",Irish,Butter
1,Kerrygold is not a dairy in Ireland. It is jus...,Irish,Butter
2,This is an excellent butter for eating but ter...,Irish,Butter
3,I purchased an 8 oz at the local Kroger for 3....,Irish,Butter
4,And I'm picky about the dairy I use. save your...,Irish,Butter


In [4]:
train.shape

(50, 3)

In [5]:
train['word_count'] = train['Review'].apply(lambda x: len(str(x).split(" ")))

In [6]:
# adding a column for the number of words of the review
train[['Review','word_count']].head()

,Review,word_count
0,"I didn’t get the flavor I was expecting, espec...",19
1,Kerrygold is not a dairy in Ireland. It is jus...,75
2,This is an excellent butter for eating but ter...,34
3,I purchased an 8 oz at the local Kroger for 3....,38
4,And I'm picky about the dairy I use. save your...,24


In [7]:
train['char_count'] = train['Review'].str.len() 

In [8]:
# Adding a column for the number of characters of the review. This also includes spaces

train[['Review','char_count']].head()

,Review,char_count
0,"I didn’t get the flavor I was expecting, espec...",97
1,Kerrygold is not a dairy in Ireland. It is jus...,419
2,This is an excellent butter for eating but ter...,181
3,I purchased an 8 oz at the local Kroger for 3....,199
4,And I'm picky about the dairy I use. save your...,115


In [9]:
train.head(5)

,Review,Brand,Category,word_count,char_count
0,"I didn’t get the flavor I was expecting, espec...",Irish,Butter,19,97
1,Kerrygold is not a dairy in Ireland. It is jus...,Irish,Butter,75,419
2,This is an excellent butter for eating but ter...,Irish,Butter,34,181
3,I purchased an 8 oz at the local Kroger for 3....,Irish,Butter,38,199
4,And I'm picky about the dairy I use. save your...,Irish,Butter,24,115


#### Average word

In [10]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))


In [11]:
train['avg_word'] = train['Review'].apply(lambda x: avg_word(x))

In [12]:
train[['Review','avg_word']].head()

,Review,avg_word
0,"I didn’t get the flavor I was expecting, espec...",4.157895
1,Kerrygold is not a dairy in Ireland. It is jus...,4.600000
2,This is an excellent butter for eating but ter...,4.352941
3,I purchased an 8 oz at the local Kroger for 3....,4.263158
4,And I'm picky about the dairy I use. save your...,3.833333


#### Natural Language Processing

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kftsu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

This means there are stopwords in the reviews.
I wanted to look at the stopwords.

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [15]:
#Counting the number of stop words

train['stopwords'] = train['Review'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['Review','stopwords']].head()

,Review,stopwords
0,"I didn’t get the flavor I was expecting, espec...",7
1,Kerrygold is not a dairy in Ireland. It is jus...,30
2,This is an excellent butter for eating but ter...,14
3,I purchased an 8 oz at the local Kroger for 3....,12
4,And I'm picky about the dairy I use. save your...,12


In [16]:
#Counting number of hashtags present in the review

train['hastags'] = train['Review'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['Review','hastags']].head()

,Review,hastags
0,"I didn’t get the flavor I was expecting, espec...",0
1,Kerrygold is not a dairy in Ireland. It is jus...,0
2,This is an excellent butter for eating but ter...,0
3,I purchased an 8 oz at the local Kroger for 3....,0
4,And I'm picky about the dairy I use. save your...,0


In [17]:
# Counting the numerics in the review

train['numerics'] = train['Review'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train[['Review','numerics']].head()

,Review,numerics
0,"I didn’t get the flavor I was expecting, espec...",0
1,Kerrygold is not a dairy in Ireland. It is jus...,0
2,This is an excellent butter for eating but ter...,0
3,I purchased an 8 oz at the local Kroger for 3....,1
4,And I'm picky about the dairy I use. save your...,0


In [18]:
# number of uppercases present in reviews

train['upper'] = train['Review'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
train[['Review','upper']].head()

,Review,upper
0,"I didn’t get the flavor I was expecting, espec...",3
1,Kerrygold is not a dairy in Ireland. It is jus...,1
2,This is an excellent butter for eating but ter...,1
3,I purchased an 8 oz at the local Kroger for 3....,2
4,And I'm picky about the dairy I use. save your...,1


I wanted to transform uppercases to lowercase to avoid having multiple copies of the same words.

In [19]:
#transforming uppercases to lowercase

train['Review'] = train['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['Review'].head()

0    i didn’t get the flavor i was expecting, espec...
1    kerrygold is not a dairy in ireland. it is jus...
2    this is an excellent butter for eating but ter...
3    i purchased an 8 oz at the local kroger for 3....
4    and i'm picky about the dairy i use. save your...
Name: Review, dtype: object

I wanted to remove special characters in the reviews.

In [20]:
#removing special characters

train['Review'] = train['Review'].str.replace('[^\w\s]','')
train['Review'].head()

C:\Users\kftsu\AppData\Local\Temp\ipykernel_25032\3484550088.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Review'] = train['Review'].str.replace('[^\w\s]','')


0    i didnt get the flavor i was expecting especia...
1    kerrygold is not a dairy in ireland it is just...
2    this is an excellent butter for eating but ter...
3    i purchased an 8 oz at the local kroger for 39...
4    and im picky about the dairy i use save your m...
Name: Review, dtype: object

I wanted to make a list of stopwords so I can remove them.

In [21]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
train['Review'] = train['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['Review'].head()

0    didnt get flavor expecting especially price wa...
1    kerrygold dairy ireland umbrella marketing bra...
2    excellent butter eating terrible baking makes ...
3    purchased 8 oz local kroger 399 also purchased...
4    im picky dairy use save money good cheaper eve...
Name: Review, dtype: object

I wanted to remove the common words

In [22]:
#count of common words
freq = pd.Series(' '.join(train['Review']).split()).value_counts()[:10]
freq

butter       53
great        15
taste        15
good         10
like          9
use           8
well          8
love          7
kerrygold     7
brand         7
dtype: int64

In [23]:
#Removing the most common words

freq = list(freq.index)
train['Review'] = train['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['Review'].head()

0    didnt get flavor expecting especially price wa...
1    dairy ireland umbrella marketing brandname man...
2    excellent eating terrible baking makes cookies...
3    purchased 8 oz local kroger 399 also purchased...
4        im picky dairy save money cheaper even better
Name: Review, dtype: object

We can see that the common words are removed as their presence will not of any use in classification of my text data.

Next, I wanted to remove the rare words. This is because the association between these rare words and other words is dominated by noise.

In [24]:
freq = pd.Series(' '.join(train['Review']).split()).value_counts()[-10:]
freq

reading     1
tried       1
today       1
lol         1
problem     1
italian     1
cheeseby    1
anymore     1
probobly    1
result      1
dtype: int64

In [25]:
#Removing the most rare words

freq = list(freq.index)
train['Review'] = train['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['Review'].head()

0    didnt get flavor expecting especially price wa...
1    dairy ireland umbrella marketing brandname man...
2    excellent eating terrible baking makes cookies...
3    purchased 8 oz local kroger 399 also purchased...
4        im picky dairy save money cheaper even better
Name: Review, dtype: object

I wanted to correct spelling of the reviews.

In [26]:
#correcting spelling

from textblob import TextBlob
train['Review'][:5].apply(lambda x: str(TextBlob(x).correct()))


0    didn get flavor expecting especially price wan...
1    dairy ireland umbrella marketing brandname man...
2    excellent eating terrible baking makes colonie...
3    purchased 8 oz local roger 399 also purchased ...
4         in pick dairy save money cheaper even better
Name: Review, dtype: object

#### Tokenization
-dividing the text into a sequence of words or sentences. (From David's lecture notes)

In [27]:
#Tokenization

nltk.download('punkt')
TextBlob(train['Review'][1]).words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kftsu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


WordList(['dairy', 'ireland', 'umbrella', 'marketing', 'brandname', 'many', 'different', 'dairies', 'europe', 'considered', 'premium', 'ordinary', 'yellower', 'color', 'american', 'butters', 'ill', 'give', 'disappointing', 'many', 'common', 'brands', 'even', 'kind', 'served', 'cafeterias', 'much', 'better', 'wont', 'purchasing'])

#### Stemming
-removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based 
approach. For this purpose, I will use PorterStemmer from the NLTK library. (From David's Lecture notes)

In [28]:
#Stemming

from nltk.stem import PorterStemmer
st = PorterStemmer()
train['Review'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0       didnt get flavor expect especi price want much
1    dairi ireland umbrella market brandnam mani di...
2    excel eat terribl bake make cooki fall apart l...
3    purchas 8 oz local kroger 399 also purchas pou...
4        im picki dairi save money cheaper even better
Name: Review, dtype: object

#### Lemmatization
-converts the word into its root word, rather than just stripping the suffices. It makes use of the vocabulary and does a morphological analysis to obtain the root word (From David's Lecture notes)

In [29]:
# Lemmatization

nltk.download('wordnet')
from textblob import Word
train['Review'] = train['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['Review'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kftsu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



**********************************************************************
  Resource omw-1.4 not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('omw-1.4')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/omw-1.4

  Searched in:
    - 'C:\\Users\\kftsu/nltk_data'
    - 'C:\\Users\\kftsu\\anaconda3\\nltk_data'
    - 'C:\\Users\\kftsu\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\kftsu\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\kftsu\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************



MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.


I wanted to extract bigrams from the reviews using the ngrams function of the textblob. I wanted to capture the language structure, like what letter or word is likely to follow the given one. I wanted to work with bigrams to capture general knowlege. (From David's lecture notes)

In [30]:
TextBlob(train['Review'][0]).ngrams(2)

[WordList(['didnt', 'get']),
 WordList(['get', 'flavor']),
 WordList(['flavor', 'expecting']),
 WordList(['expecting', 'especially']),
 WordList(['especially', 'price']),
 WordList(['price', 'wanted']),
 WordList(['wanted', 'much'])]

#### Term frequency
-the ratio of the count of a word present in a sentence, to the length of the sentence. (From David's lecture notes)

In [31]:
#term frequency

tf1 = (train['Review'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,many,2
1,dairy,1
2,butters,1
3,wont,1
4,better,1
5,much,1
6,cafeterias,1
7,served,1
8,kind,1
9,even,1


#### Inverse Document Frequency (IDF)
-the log of the ratio of the total number of rows to the 
number of rows in which that word is present. The more the value of IDF, the more unique is the word. (From David's Lecture notes)


In [32]:
#getting IDF

import numpy as np

for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['Review'].str.contains(word)])))

tf1

,words,tf,idf
0,many,2,3.912023
1,dairy,1,2.813411
2,butters,1,3.218876
3,wont,1,3.218876
4,better,1,2.525729
5,much,1,2.302585
6,cafeterias,1,3.912023
7,served,1,2.813411
8,kind,1,3.912023
9,even,1,2.525729


#### TF-IDF
-the multiplication of the TF and IDF calculated above.The importance of a term is inversely related to its frequency across documents (Capitalone.com)

In [33]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,many,2,3.912023,7.824046
1,dairy,1,2.813411,2.813411
2,butters,1,3.218876,3.218876
3,wont,1,3.218876,3.218876
4,better,1,2.525729,2.525729
5,much,1,2.302585,2.302585
6,cafeterias,1,3.912023,3.912023
7,served,1,2.813411,2.813411
8,kind,1,3.912023,3.912023
9,even,1,2.525729,2.525729


#### Bag of Words (BoW)
-representation of text which describes the presence of words within the text data. The intuition behind this is that two similar text fields will contain similar kind of words, and will therefore have a similar bag of words. (From David's Lecture notes)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(train['Review'])

train_vect

<50x348 sparse matrix of type '<class 'numpy.float64'>'
	with 483 stored elements in Compressed Sparse Row format>

In [35]:
print(train_vect)

  (0, 330)	0.4316670407335372
  (0, 239)	0.4316670407335372
  (0, 107)	0.4316670407335372
  (0, 113)	0.3903744715252307
  (0, 130)	0.31978435821471934
  (0, 88)	0.4316670407335372
  (1, 246)	0.2076146568732704
  (1, 340)	0.18775457542477667
  (1, 29)	0.16273383186115575
  (1, 46)	0.2076146568732704
  (1, 278)	0.2076146568732704
  (1, 174)	0.2076146568732704
  (1, 36)	0.2076146568732704
  (1, 60)	0.2076146568732704
  (1, 92)	0.2076146568732704
  (1, 161)	0.2076146568732704
  (1, 42)	0.18775457542477667
  (1, 16)	0.18775457542477667
  (1, 58)	0.16273383186115575
  (1, 345)	0.2076146568732704
  (1, 220)	0.2076146568732704
  (1, 237)	0.2076146568732704
  (1, 65)	0.2076146568732704
  (1, 109)	0.2076146568732704
  (1, 79)	0.2076146568732704
  :	:
  (43, 318)	0.26957497045151596
  (43, 243)	0.22702608343277972
  (44, 238)	0.5773502691896257
  (44, 7)	0.5773502691896257
  (44, 191)	0.5773502691896257
  (45, 282)	0.7556627889402567
  (45, 29)	0.6549608762445532
  (46, 106)	0.4257240111788281
  

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train['Review'])
train_bow

<50x392 sparse matrix of type '<class 'numpy.int64'>'
	with 562 stored elements in Compressed Sparse Row format>

In [37]:
print(train_bow)

  (0, 97)	1
  (0, 164)	1
  (0, 147)	1
  (0, 128)	1
  (0, 118)	1
  (0, 272)	1
  (0, 370)	1
  (0, 236)	1
  (1, 236)	1
  (1, 89)	1
  (1, 191)	1
  (1, 364)	1
  (1, 228)	1
  (1, 41)	1
  (1, 226)	2
  (1, 98)	1
  (1, 88)	1
  (1, 120)	1
  (1, 73)	1
  (1, 270)	1
  (1, 253)	1
  (1, 388)	1
  (1, 66)	1
  (1, 18)	1
  (1, 48)	1
  :	:
  (44, 271)	1
  (45, 35)	1
  (45, 320)	1
  (46, 77)	1
  (46, 109)	1
  (46, 225)	1
  (46, 275)	1
  (46, 11)	1
  (46, 116)	1
  (48, 39)	1
  (48, 146)	1
  (48, 357)	1
  (48, 50)	1
  (48, 55)	1
  (49, 66)	1
  (49, 15)	1
  (49, 86)	1
  (49, 324)	1
  (49, 240)	1
  (49, 387)	1
  (49, 281)	1
  (49, 383)	1
  (49, 290)	1
  (49, 28)	1
  (49, 377)	1


#### Sentiment Analysis

In [38]:
train['Review'][:5].apply(lambda x: TextBlob(x).sentiment)

0                                     (0.1, 0.6)
1     (0.045000000000000005, 0.5700000000000001)
2      (0.09999999999999999, 0.7666666666666666)
3    (-0.03714285714285713, 0.22000000000000003)
4                                     (0.5, 0.5)
Name: Review, dtype: object

In [39]:
train['sentiment'] = train['Review'].apply(lambda x: TextBlob(x).sentiment[0] )
train[['Review','sentiment']].head()

,Review,sentiment
0,didnt get flavor expecting especially price wa...,0.100000
1,dairy ireland umbrella marketing brandname man...,0.045000
2,excellent eating terrible baking makes cookies...,0.100000
3,purchased 8 oz local kroger 399 also purchased...,-0.037143
4,im picky dairy save money cheaper even better,0.500000


In [40]:
train

,Review,Brand,Category,word_count,char_count,avg_word,stopwords,hastags,numerics,upper,sentiment
0,didnt get flavor expecting especially price wa...,Irish,Butter,19,97,4.157895,7,0,0,3,0.100000
1,dairy ireland umbrella marketing brandname man...,Irish,Butter,75,419,4.600000,30,0,0,1,0.045000
2,excellent eating terrible baking makes cookies...,Irish,Butter,34,181,4.352941,14,0,0,1,0.100000
3,purchased 8 oz local kroger 399 also purchased...,Irish,Butter,38,199,4.263158,12,0,1,2,-0.037143
4,im picky dairy save money cheaper even better,Irish,Butter,24,115,3.833333,12,0,0,1,0.500000
5,drastically priced,Irish,Butter,3,23,7.000000,1,0,0,0,0.000000
6,dont know pricey,Irish,Butter,11,54,4.000000,6,0,0,1,0.000000
7,ive read 90 grass fed finished soy corn women ...,Irish,Butter,38,218,4.763158,13,0,0,0,-0.312500
8,creamy smooth texture nice,Irish,Butter,6,38,5.500000,0,0,0,0,0.500000
9,tastes creamer get buying dealer found costco ...,Irish,Butter,24,130,4.458333,11,0,0,0,0.000000


In [41]:
#adding polarity rating

train['Polarity_Rating'] = train['sentiment'].apply(lambda x: 'Positive' if x > 0 else('Neutral' if x == 0 else 'Negative'))

In [42]:
train

,Review,Brand,Category,word_count,char_count,avg_word,stopwords,hastags,numerics,upper,sentiment,Polarity_Rating
0,didnt get flavor expecting especially price wa...,Irish,Butter,19,97,4.157895,7,0,0,3,0.100000,Positive
1,dairy ireland umbrella marketing brandname man...,Irish,Butter,75,419,4.600000,30,0,0,1,0.045000,Positive
2,excellent eating terrible baking makes cookies...,Irish,Butter,34,181,4.352941,14,0,0,1,0.100000,Positive
3,purchased 8 oz local kroger 399 also purchased...,Irish,Butter,38,199,4.263158,12,0,1,2,-0.037143,Negative
4,im picky dairy save money cheaper even better,Irish,Butter,24,115,3.833333,12,0,0,1,0.500000,Positive
5,drastically priced,Irish,Butter,3,23,7.000000,1,0,0,0,0.000000,Neutral
6,dont know pricey,Irish,Butter,11,54,4.000000,6,0,0,1,0.000000,Neutral
7,ive read 90 grass fed finished soy corn women ...,Irish,Butter,38,218,4.763158,13,0,0,0,-0.312500,Negative
8,creamy smooth texture nice,Irish,Butter,6,38,5.500000,0,0,0,0,0.500000,Positive
9,tastes creamer get buying dealer found costco ...,Irish,Butter,24,130,4.458333,11,0,0,0,0.000000,Neutral


In [43]:
train= train[['Review', 'Polarity_Rating']]
train.head()

,Review,Polarity_Rating
0,didnt get flavor expecting especially price wa...,Positive
1,dairy ireland umbrella marketing brandname man...,Positive
2,excellent eating terrible baking makes cookies...,Positive
3,purchased 8 oz local kroger 399 also purchased...,Negative
4,im picky dairy save money cheaper even better,Positive


In [44]:
train.shape

(50, 2)

I wanted to apply One hot encoding on negative, neutral, and positive

In [45]:
one_hot = pd.get_dummies(train["Polarity_Rating"])
train.drop(['Polarity_Rating'],axis=1,inplace=True)
train = pd.concat([train,one_hot],axis=1)
train.head()

C:\Users\kftsu\AppData\Local\Temp\ipykernel_25032\1597170316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['Polarity_Rating'],axis=1,inplace=True)


,Review,Negative,Neutral,Positive
0,didnt get flavor expecting especially price wa...,0,0,1
1,dairy ireland umbrella marketing brandname man...,0,0,1
2,excellent eating terrible baking makes cookies...,0,0,1
3,purchased 8 oz local kroger 399 also purchased...,1,0,0
4,im picky dairy save money cheaper even better,0,0,1


Applying Train Test Split

In [46]:
X = train['Review'].values
y = train.drop('Review', axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

Applying Vectorization

In [47]:
vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

Applying frequency, inverse document frequency

In [48]:
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train = X_train.toarray()
X_test = X_test.toarray()

Adding different layers

In [51]:
model = Sequential()

model.add(Dense(units=12673,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=4000,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=500,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=3, activation='softmax'))

opt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

Fitting the Model

In [52]:
model.fit(x=X_train, y=y_train, batch_size=256, epochs=100, validation_data=(X_test, y_test), verbose=1, callbacks=early_stop)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.0991 - accuracy: 0.3143 - val_loss: 0.9094 - val_accuracy: 0.6000
Epoch 2/100
1/1 [==============================] - 0s 316ms/step - loss: 0.9141 - accuracy: 0.6000 - val_loss: 0.9050 - val_accuracy: 0.6000
Epoch 3/100
1/1 [==============================] - 0s 310ms/step - loss: 0.8143 - accuracy: 0.6000 - val_loss: 0.9105 - val_accuracy: 0.6000
Epoch 4/100
1/1 [==============================] - 0s 315ms/step - loss: 0.7243 - accuracy: 0.6000 - val_loss: 0.8329 - val_accuracy: 0.6000
Epoch 5/100
1/1 [==============================] - 0s 346ms/step - loss: 0.5480 - accuracy: 0.6000 - val_loss: 0.7847 - val_accuracy: 0.7333
Epoch 6/100
1/1 [==============================] - 0s 348ms/step - loss: 0.4243 - accuracy: 0.9429 - val_loss: 0.7267 - val_accuracy: 0.8000
Epoch 7/100
1/1 [==============================] - 0s 313ms/step - loss: 0.3056 - accuracy: 1.0000 - val_loss: 0.7601 - val_accuracy: 0.8000
Epoch 8/100
1/1 

Evaluating of Model

In [53]:
model_score = model.evaluate(X_test, y_test, batch_size=64, verbose=1)
print('Test accuracy:', model_score[1])

1/1 [==============================] - 0s 33ms/step - loss: 0.8796 - accuracy: 0.8000
Test accuracy: 0.800000011920929


Prediction

In [54]:
preds = model.predict(X_test)
preds

1/1 [==============================] - 0s 79ms/step


array([[2.19039038e-01, 4.71324563e-01, 3.09636444e-01],
       [9.76911187e-02, 5.76400459e-01, 3.25908363e-01],
       [2.19039038e-01, 4.71324563e-01, 3.09636444e-01],
       [3.86585146e-02, 6.48557246e-02, 8.96485746e-01],
       [1.80606835e-03, 1.52789173e-03, 9.96666014e-01],
       [9.07197129e-03, 9.65929218e-03, 9.81268704e-01],
       [3.70832682e-02, 8.24466199e-02, 8.80470157e-01],
       [6.53605210e-03, 6.78230682e-03, 9.86681581e-01],
       [9.07197129e-03, 9.65929218e-03, 9.81268704e-01],
       [1.30116090e-03, 8.54282989e-04, 9.97844577e-01],
       [2.45349333e-02, 2.97011379e-02, 9.45763886e-01],
       [1.55700576e-02, 2.31736731e-02, 9.61256325e-01],
       [1.13350805e-02, 7.64023559e-03, 9.81024742e-01],
       [3.05694179e-03, 3.13306623e-03, 9.93809998e-01],
       [2.03211587e-02, 1.92775391e-02, 9.60401297e-01]], dtype=float32)

In [55]:
preds.shape

(15, 3)